## Untested model generated by ChatGPT

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Enable GPU usage
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

# Load dataset
CSV_PATH = "dataset_labels.csv"  # Update with actual path
df = pd.read_csv(CSV_PATH)

# Convert paths to absolute paths
df["image_path"] = df["image_path"].apply(lambda x: str(pathlib.Path("images") / x))
df["label"] = df["label"].astype(float)  # Convert labels to float

In [ ]:
# Split into training (80%) and testing (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Function to load and preprocess images
def load_image(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224]) / 255.0  # Normalize
    return img, label

In [ ]:
# Create TensorFlow datasets
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE


In [ ]:
def prepare_dataset(df):
    dataset = tf.data.Dataset.from_tensor_slices((df["image_path"].values, df["label"].values))
    dataset = dataset.map(load_image, num_parallel_calls=AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(df)).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

train_ds = prepare_dataset(train_df)
test_ds = prepare_dataset(test_df)

In [ ]:
# Build CNN Model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')  # Output layer with ReLU for positive values
])

In [ ]:
# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

In [ ]:
# Train Model
EPOCHS = 50
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

In [ ]:
# Plot Training History
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Save Model
model.save("uav_image_regression_model.h5")